In [7]:
from main import generate_estimation
from math import ceil
import time
import json
from datetime import datetime
import threading
# _, numOfCars = generate_estimation()
# print(_)
# numOfCars
# greenTime = min(numOfCars * 2 + 3, 120)
# print(numOfCars, greenTime)

In [8]:
class TrafficLight:
    def __init__(self, trafficID, lanes):
        self.trafficID = trafficID
        self.lanes = lanes
        self.totalCars = 0
        self.carsPerLane = 0
        self.greenTime = 0

    def update_traffic(self, totalCars):
        self.totalCars = totalCars
        self.carsPerLane = ceil(totalCars / self.lanes)
        # self.greenTime = min(self.carsPerLane * 2 + 3, 50)
        avgTimePerCar = 2.5
        max_time = 50
        min_time = 5
        self.greenTime = max(min(ceil((self.totalCars * avgTimePerCar) / (self.lanes + 1)), max_time), min_time)
    def getGreenTime(self):
        return self.greenTime
    def getTrafficID(self):
        return self.trafficID
    def resetGreenTime(self):
        self.greenTime = 0
    def __str__(self):
        return (f"Traffic Light {self.trafficID} -> Lanes: {self.lanes}, Total Cars: {self.totalCars}, "
                f"Cars per Lane: {self.carsPerLane}, Green Time: {self.greenTime}s")
       # totalCars = 20  # Simulated number of cars

class Intersection:
    def __init__(self, intersectionID, trafficLights):
        self.intersectionID = intersectionID
        self.trafficLights = trafficLights
    def getIntersectionID(self):
        return self.intersectionID
    def getTrafficLights(self):
        return self.trafficLights
    def getSignalOn(self):
        for light in self.trafficLights:
            if light.greenTime > 0:
                return light.getTrafficID
        return None
    def getTrafficLightwithID(self, trafficID):
        for light in self.trafficLights:
            if light.getTrafficID() == trafficID:
                return light
        return None


class CircularTrafficLights:
    def __init__(self, lights):
        self.lights = lights
        self.index = 0

    def __iter__(self):
        return self

    def __next__(self):
        if not self.lights:
            raise StopIteration
        light = self.lights[self.index]
        self.index = (self.index + 1) % len(self.lights)
        return light

    # def update_light(self, trafficID, totalCars, carsPerLane, greenTime):
    #     for light in self.lights:
    #         if light.trafficID == trafficID:
    #             light.update_traffic(totalCars, carsPerLane, greenTime)
    #             print(f"Updated Traffic Light {trafficID}")

def save_to_database(data, filename='test.json'):
    try:
        with open(filename, 'r') as json_file:
            existing_data = json.load(json_file)
            if not isinstance(existing_data, list):
                existing_data = []
    except (FileNotFoundError, json.JSONDecodeError):
        existing_data = []

    
    data_dict = {
        f"Cycle {data[0]}": {
            "trafficID": data[1],
            "num_cars": data[2],
            "weatherStamp:": data[3],
            "timeStamp": data[4],
            "dateStamp": data[5],
            "greenTime": data[6]
        }
    }

    existing_data.append(data_dict)  # Use append to add the dictionary to the list

    with open(filename, 'w') as json_file:
        json.dump(existing_data, json_file, indent=4)

In [5]:
import sqlite3
from datetime import datetime

# Assuming main.py is modified to return structured data
from main import generate_estimation

# Database setup
conn = sqlite3.connect('traffic_data.db')
c = conn.cursor()

# Create tables if they don't exist
c.execute('''
CREATE TABLE IF NOT EXISTS intersections (
    IntersectionID INTEGER PRIMARY KEY,
    IntersectionName TEXT,
    Location TEXT
)''')

c.execute('''
CREATE TABLE IF NOT EXISTS signals (
    SignalID INTEGER PRIMARY KEY,
    IntersectionID INTEGER,
    SignalName TEXT,
    SignalPosition TEXT,
    FOREIGN KEY (IntersectionID) REFERENCES intersections (IntersectionID)
)''')

c.execute('''
CREATE TABLE IF NOT EXISTS traffic_flows (
    FlowID INTEGER PRIMARY KEY,
    SignalID INTEGER,
    TotalCarsFlowingOut INTEGER,
    TotalAssignedGreenTime INTEGER,
    FOREIGN KEY (SignalID) REFERENCES signals (SignalID)
)''')

c.execute('''
CREATE TABLE IF NOT EXISTS directed_flows (
    DirectedFlowID INTEGER PRIMARY KEY,
    FlowID INTEGER,
    Direction TEXT,
    DirectedFlow INTEGER,
    Destination TEXT,
    ExpectedTravelTime INTEGER,
    NeededGreenTime INTEGER, 
    Time TEXT,  
    Date TEXT,
    FOREIGN KEY (FlowID) REFERENCES traffic_flows (FlowID)
)''')

In [9]:
# Example usage:
import random
lights = [
    TrafficLight(1, 4), # trafficID, lanes
    TrafficLight(2, 4),
    TrafficLight(3, 4),
    TrafficLight(4, 4)
]

traffic_circle = CircularTrafficLights(lights)


# Simulating updates and looping through them
i = 0
cycle = 0
j=0
intersections = ['A','B']
for light in traffic_circle:
    
    intersection = intersections[j]
     
    if(i % len(lights) == 0):
        cycle += 1
    if(cycle == 11):
        break
    i += 1
    time.sleep(2)

    history, numOfCars = generate_estimation()
    
    light.update_traffic(numOfCars)
    data = [cycle, light.trafficID, numOfCars, history[0][2], history[0][3], history[0][4], light.getGreenTime(), intersection]
    save_to_database(data)
    print(light)



     # MULTI NETWORK DATA GATHERING
    # Inserting formatted data into new structured database
    forward_flow = int(numOfCars * 0.7)
    left_flow = int(numOfCars * 0.2)
    right_flow = int(numOfCars * 0.1)

    # Dates and time
    # recordedDay = history[0][5]
    recordedDate = history[0][4]
    recordedTime = history[0][3]

    '''ASSUMPTIONS'''
    roadLength = 1 # road length in km
    destination = data[7]
    estimation = history[0][2]
    different_road_speeds = [80, 90, 100] # for randomizing road speeds
    V_max = random.choice(different_road_speeds) # Max speed of road segment

    flows = [forward_flow, left_flow, right_flow]
    assumptions = [0.7, 0.2, 0.1] # assuming 70% go forward, 20% left and 10% right
    travelTimes = [] # to store trave times of the directed flows
    greenTimesPerDirectedFlow = [] # to store green times of each flow 
    for density, percent in zip(flows, assumptions):
        if density < 20:
            weight = 0.9
        elif density > 20 and density < 50:
            weight = 0.7
        elif density > 50 and density < 100:
            weight = 0.4
        else:
            weight = 0.2
    
        V_avg = V_max * weight # average car speed in the road segment
        T_seconds = (roadLength / V_avg) * 60 * 60 # in seconds
        travelTimes.append(T_seconds)
        greenTimesPerDirectedFlow.append(round(estimation*percent))  #calculating the green time where percent is the assumed % of cars in this direction and density is the


    # Insert data into database
    c.execute('INSERT INTO traffic_flows (SignalID, TotalCarsFlowingOut, TotalAssignedGreenTime) VALUES (?, ?, ?)',
                (1, numOfCars, 120))  
    flow_id = c.lastrowid
    
    # Insert directed flows
    c.execute('INSERT INTO directed_flows (FlowID, Direction, DirectedFlow, Destination, ExpectedTravelTime, NeededGreenTime, Time, Date) VALUES (?, ?, ?, ?, ?, ?, ?, ?)',
                (flow_id, 'forward', forward_flow, destination, travelTimes[0], greenTimesPerDirectedFlow[0],  recordedTime, recordedDate))
    c.execute('INSERT INTO directed_flows (FlowID, Direction, DirectedFlow, Destination, ExpectedTravelTime, NeededGreenTime, Time, Date) VALUES (?, ?, ?, ?, ?, ?, ?, ?)',
                (flow_id, 'left', left_flow, destination, travelTimes[1], greenTimesPerDirectedFlow[1], recordedTime, recordedDate))
    c.execute('INSERT INTO directed_flows (FlowID, Direction, DirectedFlow, Destination, ExpectedTravelTime, NeededGreenTime, Time, Date) VALUES (?, ?, ?, ?, ?, ?, ?, ?)',
                (flow_id, 'right', right_flow, destination, travelTimes[2], greenTimesPerDirectedFlow[2], recordedTime, recordedDate))
    
    conn.commit()




    time.sleep(light.getGreenTime())

Traffic Light 1 -> Lanes: 4, Total Cars: 126, Cars per Lane: 32, Green Time: 50s
Traffic Light 2 -> Lanes: 4, Total Cars: 103, Cars per Lane: 26, Green Time: 50s
Traffic Light 3 -> Lanes: 4, Total Cars: 150, Cars per Lane: 38, Green Time: 50s
Traffic Light 4 -> Lanes: 4, Total Cars: 70, Cars per Lane: 18, Green Time: 35s
Traffic Light 1 -> Lanes: 4, Total Cars: 16, Cars per Lane: 4, Green Time: 8s
Traffic Light 2 -> Lanes: 4, Total Cars: 64, Cars per Lane: 16, Green Time: 32s
Traffic Light 3 -> Lanes: 4, Total Cars: 34, Cars per Lane: 9, Green Time: 17s
Traffic Light 4 -> Lanes: 4, Total Cars: 109, Cars per Lane: 28, Green Time: 50s
Traffic Light 1 -> Lanes: 4, Total Cars: 61, Cars per Lane: 16, Green Time: 31s
Traffic Light 2 -> Lanes: 4, Total Cars: 110, Cars per Lane: 28, Green Time: 50s
Traffic Light 3 -> Lanes: 4, Total Cars: 52, Cars per Lane: 13, Green Time: 26s
Traffic Light 4 -> Lanes: 4, Total Cars: 117, Cars per Lane: 30, Green Time: 50s
Traffic Light 1 -> Lanes: 4, Total Ca

In [4]:
import threading
import time

def function1(arg):
    for i in range(5):
        print(f"Thread {arg} iteration {i}")
        # time.sleep(1)

# Create threads targeting the same function but with different arguments
thread1 = threading.Thread(target=function1, args=("1",))
thread2 = threading.Thread(target=function1, args=("2",))

# Start threads
thread1.start()
thread2.start()

# Wait for both threads to complete
thread1.join()
thread2.join()

# Now 'data' is a Python dictionary that contains the data from the JSON file
print(data)

Thread 1 iteration 0
Thread 1 iteration 1
Thread 1 iteration 2
Thread 1 iteration 3
Thread 1 iteration 4
Thread 2 iteration 0
Thread 2 iteration 1
Thread 2 iteration 2
Thread 2 iteration 3
Thread 2 iteration 4
Both threads have completed their execution
